In [1]:
!pip install spacy nltk pandas tqdm inflect
!python -m spacy download en_core_web_sm
import nltk
nltk.download("wordnet")
nltk.download("omw-1.4")

  Using cached nltk-3.9.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached inflect-7.5.0-py3-none-any.whl.metadata (24 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached typeguard-4.4.4-py3-none-any.whl.metadata (3.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 595.4 kB/s  0:00:55 eta 0:00:010:00:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 835.9/835.9 kB 575.9 kB/s  0:00:01572.2 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 587.9 kB/s  0:00:010.8 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 576.6 kB/s  0:00:06554.5 kB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 543.5 kB/s  0:00:20 eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 551.1 kB/s  0:00:30 eta 0:00:010:00:02
Using cached nltk-3.9.2-py3-none-any.whl (1.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9

[nltk_data] Downloading package wordnet to /home/taha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/taha/nltk_data...


True

In [2]:
import os
import json
import random
import re
from pathlib import Path
import pandas as pd
import spacy
from nltk.corpus import wordnet as wn
import nltk
from tqdm import tqdm
nlp = spacy.load("en_core_web_sm")
import inflect

In [3]:
# Initialize the inflect engine (for English singular/plural detection)
p = inflect.engine()

In [ ]:
GERMAN_COMPLEX_PATTERNS = {
    'premises': {
        'all': [
            {'tpl': 'Alle {a_pl} sind {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Alle {a_pl} gehören zur Klasse {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ohne Ausnahme sind alle {a_pl} {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Jedes {a_sg} ist {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Jeder einzelne {a_sg} gehört zur Kategorie {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Alles, was {a_sg_art} ist, ist {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Wenn etwas {a_sg_art} ist, dann ist es auch {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Die Klasse der {a_pl} ist eine Teilmenge von {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Sämtliche {a_pl} werden als {b_pl} eingestuft.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es gibt kein {a_sg}, das nicht {b_sg_art} ist.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Wer {a_sg_art} ist, qualifiziert sich als {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Alle Vertreter von {a_pl} zählen zu {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Jedes {a_sg} ist notwendigerweise {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': '{a_pl} sind ausnahmslos {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Die Gesamtheit der {a_pl} fällt unter {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'some': [
            {'tpl': 'Einige {a_pl} sind {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es gibt einige {a_pl}, die {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Mindestens ein {a_sg} ist {b_sg}.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'In manchen Fällen ist {a_sg_art} {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Einige wenige {a_pl} sind tatsächlich {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Bestimmte {a_pl} weisen Merkmale von {b_pl} auf.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es existieren {a_pl}, die auch {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ein Teil der {a_pl} gehört zu {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Man kann {a_pl} finden, die {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es kommt vor, dass {a_sg_art} {b_sg_art} ist.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Nicht alle, aber doch einige {a_pl} sind {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Gelegentlich ist ein {a_sg} auch {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Es gibt Instanzen von {a_pl}, die als {b_pl} gelten.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Eine gewisse Anzahl an {a_pl} sind {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Zumindest manche {a_pl} sind in {b_pl} enthalten.', 'a_type': 'plural', 'b_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Keine {a_pl} sind {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es gibt keine {a_pl}, die {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Kein {a_sg} kann als {b_sg_art} gelten.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Nichts, das {a_sg_art} ist, ist {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Überhaupt keine {a_pl} sind {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Niemals ist ein {a_sg} zugleich {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Die Mengen {a_pl} und {b_pl} sind disjunkt.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es ist ausgeschlossen, dass {a_sg_art} {b_sg_art} ist.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'Keinerlei {a_pl} gehören zur Klasse {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ein {a_sg} ist unter keinen Umständen {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Es existiert keine Überschneidung zwischen {a_pl} und {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Wenn etwas {a_sg_art} ist, kann es nicht {b_sg_art} sein.', 'a_type': 'singular_art', 'b_type': 'singular_art'},
            {'tpl': 'In der Gruppe der {a_pl} findet man kein einziges {b_sg}.', 'a_type': 'plural', 'b_type': 'singular'},
            {'tpl': 'Es ist falsch, dass irgendein {a_sg} {b_sg_art} ist.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Absolut kein {a_sg} ist {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'}
        ],
        'some_not': [
            {'tpl': 'Einige {a_pl} sind nicht {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Nicht jedes {a_sg} ist {b_sg_art}.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Es gibt mindestens ein {a_sg}, das nicht {b_sg_art} ist.', 'a_type': 'singular', 'b_type': 'singular_art'},
            {'tpl': 'Mindestens einige {a_pl} sind nicht {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es gibt {a_pl}, die nicht {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Manche {a_pl} fallen nicht unter die Kategorie {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Ein Teil der {a_pl} ist von {b_pl} ausgeschlossen.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es ist nicht der Fall, dass alle {a_pl} {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Bestimmte {a_pl} sind keine {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Man findet {a_pl}, die nicht {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Es existieren Exemplare von {a_pl}, die keine {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Nicht alle Vertreter von {a_pl} sind {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'},
            {'tpl': 'Einige {a_pl} besitzen die Eigenschaft {b_sg} nicht.', 'a_type': 'plural', 'b_type': 'singular'},
            {'tpl': 'Es gibt Fälle, in denen ein {a_sg} kein {b_sg} ist.', 'a_type': 'singular', 'b_type': 'singular'},
            {'tpl': 'Ein gewisser Anteil an {a_pl} ist nicht {b_pl}.', 'a_type': 'plural', 'b_type': 'plural'}
        ]
    },
    'conclusions': [
        'Daher {conclusion}.',
        'Folglich {conclusion}.',
        'Somit {conclusion}.',
        'Aus diesen Gründen {conclusion}.',
        'Demnach {conclusion}.',
        'Man kann daraus schließen, dass {conclusion}.',
        'Infolgedessen {conclusion}.',
        'Das bedeutet, dass {conclusion}.',
        'Daraus ergibt sich: {conclusion}.',
        'Zusammenfassend lässt sich sagen, dass {conclusion}.'
    ]
}

In [ ]:
GERMAN_COMPLEX_PATTERNS_WITH_FOURTH_VAR = {
    'premises': {
        'all': [
            {'tpl': 'Jedes {d_sg}, das {a_sg_art} ist, ist auch {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Wenn {d_sg_art} {a_sg_art} ist, dann muss es {b_sg_art} sein.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Alle {d_pl}, die {a_pl} sind, sind ebenfalls {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Ohne Ausnahme gelten die {d_pl}, die {a_pl} sind, als {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Jedes einzelne {d_sg} in {a_sg_art} ist {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Jeder {d_sg}, der als {a_sg_art} gilt, ist notwendigerweise {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Für jedes {d_sg}, das zugleich {a_sg_art} ist, gilt, dass es {b_sg_art} ist.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Alles aus {d_pl}, was {a_sg_art} ist, gehört auch zu {b_pl}.', 'a_type': 'singular_art', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Wenn ein {d_sg} {a_sg_art} ist, folgt daraus, dass es {b_sg_art} ist.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Die Gesamtheit der {d_pl} mit der Eigenschaft {a_pl} ist in {b_pl} enthalten.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Innerhalb von {d_pl} sind alle {a_pl} zugleich {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Wer in {d_pl} {a_sg_art} ist, ist immer auch {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'plural'},
            {'tpl': 'Kein {d_sg} ist {a_sg_art}, ohne auch {b_sg_art} zu sein.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Jedes Exemplar von {d_pl}, das unter {a_pl} fällt, ist {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es ist sicher, dass jedes {d_sg}, das {a_sg} ist, auch {b_sg} ist.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'}
        ],
        'some': [
            {'tpl': 'Es existiert mindestens ein {d_sg}, das sowohl {a_sg_art} als auch {b_sg_art} ist.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Zumindest ein {d_sg}, das {a_sg_art} ist, ist {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Einige {d_pl} sind sowohl {a_pl} als auch {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Man findet {d_pl}, die {a_pl} und zugleich {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Mindestens eines der {d_pl} ist {a_pl} und {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es gibt Fälle in {d_pl}, in denen {a_sg_art} auch {b_sg_art} ist.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'plural'},
            {'tpl': 'Bestimmte {d_pl} weisen sowohl {a_pl} als auch {b_pl} Merkmale auf.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es existieren Exemplare von {d_sg}, die {a_sg} und {b_sg} vereinen.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Unter den {d_pl} gibt es solche, die {a_pl} und {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Einige {d_pl} fallen unter die Kategorien {a_pl} und {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Man stößt auf {d_sg}, die gleichzeitig {a_sg_art} und {b_sg_art} sind.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Nicht selten ist ein {d_sg} sowohl {a_sg} als auch {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'In der Gruppe {d_pl} finden sich {a_pl}, die {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Teilweise sind {d_pl} sowohl {a_pl} als auch {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es kommen {d_pl} vor, die Merkmale von {a_pl} und {b_pl} zeigen.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'no': [
            {'tpl': 'Keines der {d_pl}, die {a_pl} sind, ist {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Nichts ist gleichzeitig {d_sg_art}, {a_sg_art} und {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Überhaupt keine {d_pl} sind {b_pl}, wenn sie {a_pl} sind.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es gibt keine {d_pl}, die sowohl {a_pl} als auch {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Nicht ein {d_sg} existiert, das {a_sg} und {b_sg} zugleich wäre.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Es ist unmöglich, dass ein {d_sg} zugleich {a_sg_art} und {b_sg_art} ist.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Kein Vertreter von {d_pl} kann sowohl {a_pl} als auch {b_pl} sein.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Innerhalb von {d_pl} schließt {a_pl} die Eigenschaft {b_pl} aus.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Wer {d_sg_art} und {a_sg_art} ist, kann niemals {b_sg_art} sein.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular_art'},
            {'tpl': 'Die Kategorien {a_pl} und {b_pl} überschneiden sich bei {d_pl} nicht.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es gibt keinerlei {d_pl}, die {a_pl} und gleichzeitig {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Niemals ist ein {d_sg} sowohl {a_sg} als auch {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Falsch ist die Annahme, ein {d_sg} könne {a_sg} und {b_sg} zugleich sein.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Kein {d_sg} vereint die Merkmale von {a_sg} und {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'In der Klasse {d_pl} gibt es keine Kombination von {a_pl} und {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ],
        'some_not': [
            {'tpl': 'Nicht jedes {d_sg}, das {a_sg_art} ist, ist {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Es gibt mindestens ein {d_sg}, das {a_sg_art} ist, aber nicht {b_sg_art}.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'Einige {d_pl}, die {a_pl} sind, sind nicht {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Mindestens einige {d_pl} sind {a_pl}, jedoch nicht {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es existiert ein {d_sg}, das {a_sg} ist, aber nicht {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Nicht alle {d_pl}, die {a_pl} sind, fallen unter {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es finden sich {d_pl}, welche {a_pl}, aber keine {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Zumindest ein {d_sg} besitzt {a_sg}, aber nicht {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Manche {d_pl} mit dem Merkmal {a_pl} sind von {b_pl} ausgeschlossen.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Es ist nicht so, dass jedes {d_sg} in {a_sg_art} auch {b_sg_art} ist.', 'a_type': 'singular_art', 'b_type': 'singular_art', 'd_type': 'singular'},
            {'tpl': 'In {d_pl} gibt es Vertreter von {a_pl}, die nicht {b_pl} sind.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Nicht jedes Exemplar von {d_pl}, das {a_pl} ist, ist auch {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Bestimmte {d_sg} sind zwar {a_sg}, jedoch nicht {b_sg}.', 'a_type': 'singular', 'b_type': 'singular', 'd_type': 'singular'},
            {'tpl': 'Es gibt {d_pl}, die {a_pl} sind und dennoch nicht {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'},
            {'tpl': 'Ein gewisser Teil der {d_pl} ist {a_pl}, aber nicht {b_pl}.', 'a_type': 'plural', 'b_type': 'plural', 'd_type': 'plural'}
        ]
    }
}

In [6]:
def get_singular_and_plural(term):
    """Return (singular, plural) using English inflect heuristics.
    If inflect cannot determine, fall back to the term itself for both.
    """
    term = term.strip()
    # Try English singular detection
    try:
        sg_attempt = p.singular_noun(term)
    except Exception:
        sg_attempt = False
    if sg_attempt is not False and sg_attempt is not None:
        sg = sg_attempt
        pl = term
    else:
        sg = term
        try:
            pl = p.plural_noun(term) or term
        except Exception:
            pl = term
    return sg, pl

In [7]:
def get_article_german(term):
    """Return a simple indefinite article in German for singular forms.
    This is heuristic: returns 'eine' if term ends with 'e' (common feminine clue), else 'ein'.
    """
    term = term.strip()
    if term.endswith('e'):
        return 'eine ' + term
    return 'ein ' + term

In [8]:
def parse_sentence_for_type_and_terms(sentence):
    sentence = sentence.strip()
    # reuse same English parsing logic (input is English)
    match = re.match(r'All (.*) that are (.*) are (.*).', sentence)
    if match: return 'all', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are not (.*).', sentence)
    if match: return 'some_not', match.group(1), match.group(2), match.group(3)
    match = re.match(r'Some (.*) that are (.*) are (.*).', sentence)
    if match: return 'some', match.group(1), match.group(2), match.group(3)
    match = re.match(r'No (.*) that are (.*) are (.*).', sentence)
    if match: return 'no', match.group(1), match.group(2), match.group(3)
    match = re.match(r'All (.*) are (.*).', sentence)
    if match: return 'all', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are not (.*).', sentence)
    if match: return 'some_not', None, match.group(1), match.group(2)
    match = re.match(r'Some (.*) are (.*).', sentence)
    if match: return 'some', None, match.group(1), match.group(2)
    match = re.match(r'No (.*) are (.*).', sentence)
    if match: return 'no', None, match.group(1), match.group(2)
    raise ValueError(f'Could not parse basic syllogism type from sentence: {sentence}')

In [9]:
def create_sub_dict(term_original_subject, term_original_predicate, term_original_fourth, synonym_dict=None):
    sub_map = {}
    sg_a, pl_a = get_singular_and_plural(term_original_subject)
    sub_map['a_sg'] = sg_a
    sub_map['a_pl'] = pl_a
    sub_map['a_sg_art'] = get_article_german(sg_a)
    sg_b, pl_b = get_singular_and_plural(term_original_predicate)
    sub_map['b_sg'] = sg_b
    sub_map['b_pl'] = pl_b
    sub_map['b_sg_art'] = get_article_german(sg_b)
    if term_original_fourth:
        sg_d, pl_d = get_singular_and_plural(term_original_fourth)
        sub_map['d_sg'] = sg_d
        sub_map['d_pl'] = pl_d
        sub_map['d_sg_art'] = get_article_german(sg_d)
    return sub_map

In [10]:
def safe_lowercase_first(s, sub_map):
    for val in sub_map.values():
        if isinstance(val, str) and s.startswith(val):
            return s
    return s[:1].lower() + s[1:]

In [11]:
def convert_syllogism_to_complex(syllogism_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    simple_syllogism = syllogism_data['syllogism']
    premises_simple, conclusion_simple = split_syllogism_into_sentences(simple_syllogism)
    complex_premises = []
    for premise in premises_simple:
        p_type, p_fourth, p_subject, p_predicate = parse_sentence_for_type_and_terms(premise)
        sub_map = create_sub_dict(p_subject, p_predicate, p_fourth)
        if p_fourth:
            p_template = random.choice(complex_patterns_data_with_fourth_var['premises'][p_type])
        else:
            p_template = random.choice(complex_patterns_data['premises'][p_type])
        p_complex = p_template['tpl'].format(**sub_map)
        p_complex = p_complex[0].upper() + p_complex[1:]
        complex_premises.append(p_complex)
    c_type, c_fourth, c_subject, c_predicate = parse_sentence_for_type_and_terms(conclusion_simple)
    c_sub_map = create_sub_dict(c_subject, c_predicate, c_fourth)
    if c_fourth:
        c_premise_template = random.choice(complex_patterns_data_with_fourth_var['premises'][c_type])
    else:
        c_premise_template = random.choice(complex_patterns_data['premises'][c_type])
    c_premise_complex = c_premise_template['tpl'].format(**c_sub_map)
    c_premise_complex = safe_lowercase_first(c_premise_complex, c_sub_map)
    c_wrapper_template = random.choice(complex_patterns_data['conclusions'])
    c_complex = c_wrapper_template.format(conclusion=c_premise_complex)
    c_complex = c_complex[:-1] if c_complex.endswith('.') else c_complex
    c_complex = c_complex[0].upper() + c_complex[1:]
    final_complex_syllogism = ' '.join(complex_premises) + ' ' + c_complex
    new_data = syllogism_data.copy()
    new_data['syllogism'] = final_complex_syllogism
    return new_data

In [12]:
def split_syllogism_into_sentences(syllogism):
    sentences = [s.strip() for s in syllogism.split('.') if s.strip()]
    if len(sentences) < 2:
        raise ValueError(f'Syllogism must contain at least one premise and one conclusion: {syllogism}')
    sentences = [s + '.' for s in sentences]
    conclusion = sentences.pop()
    premises = sentences
    return premises, conclusion

In [13]:
def process_dataset(input_data, complex_patterns_data, complex_patterns_data_with_fourth_var):
    complex_dataset = []
    for item in input_data:
        try:
            complex_item = convert_syllogism_to_complex(item, complex_patterns_data, complex_patterns_data_with_fourth_var)
            complex_dataset.append(complex_item)
        except ValueError as e:
            print(f' Überspringe Eintrag {item.get(id, 'N/A')} wegen Parsing-Fehler: {e}')
            continue
    return complex_dataset

In [14]:
# Example English input data (same style as original)
sample_data = [
    {
        'id': 'ex-1',
        'syllogism': 'All dogs are animals. All animals are living things. Therefore some dogs are living things.',
        'validity': True,
        'plausibility': True
    },
    {
        'id': 'ex-2',
        'syllogism': 'No fish are mammals. Some mammals are whales. Some whales are not fish.',
        'validity': True,
        'plausibility': True
    }
]
# Run conversion using German templates
complex_syllogism_data = process_dataset(sample_data, GERMAN_COMPLEX_PATTERNS, GERMAN_COMPLEX_PATTERNS_WITH_FOURTH_VAR)
print(json.dumps(complex_syllogism_data, indent=4, ensure_ascii=False))

 Überspringe Eintrag N/A wegen Parsing-Fehler: Could not parse basic syllogism type from sentence: Therefore some dogs are living things.
[
    {
        "id": "ex-2",
        "syllogism": "Keine fish sind mammals. In manchen Fällen ist ein mammal eine whale. Aus diesen Gründen nicht jedes whale ist ein fish.",
        "validity": true,
        "plausibility": true
    }
]
